# ***MAIN FUNCTIONALITY***

In [1]:
from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()

site = Site(latitude=54.95, longitude=-5.95)
array_1 = SolarPVArray(1, 35, 83, 25, 0.88)
pv_model = SolarPVModel(site, [array_1])

2024-02-15 23:14:03,972 - INFO - Fetching TMY data for latitude: 54.95, longitude: -5.95
2024-02-15 23:14:05,204 - INFO - HTTP Request: GET https://re.jrc.ec.europa.eu/api/tmy?lat=54.95&lon=-5.95&startyear=2005&endyear=2015&outputformat=json "HTTP/1.1 200 OK"
2024-02-15 23:14:06,389 - INFO - TMY data obtained for: 54.95, longitude: -5.95
2024-02-15 23:14:06,390 - INFO - *******************
2024-02-15 23:14:06,390 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 83deg, Lifespan: 25yrs
2024-02-15 23:14:06,391 - INFO - *******************
2024-02-15 23:14:06,391 - INFO - *******************
2024-02-15 23:14:06,391 - INFO - Starting model calculations for SolarPVModel.
2024-02-15 23:14:06,392 - INFO - *******************
2024-02-15 23:14:06,392 - INFO - Simulating model for 1kWp, 35 degrees pitch & azimuth at 83 degrees WoS
2024-02-15 23:14:06,423 - INFO - *******************
2024-02-15 23:14:06,424 - INFO - Model calculations completed.
2024-02-15 23:14:06,445 - INFO - M

In [4]:
%reset -f

# ***Example Running Model with variable inputs***

In [5]:
# %%timeit

from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()

# Set up site and obtain TMY data
site = Site(name="Test Site", address = "123 Test Street", client = "Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east = 0)

2024-02-15 23:14:13,770 - INFO - TMY data obtained for: 54.60452, longitude: -5.9286
2024-02-15 23:14:13,771 - INFO - *******************


In [6]:
pv_kwp = 1
surface_pitch = 35
surface_azimuth = -90
lifespan = 25
pv_eol_derating = 0.88

# Set up PV arrays
array_1 = SolarPVArray(pv_kwp, surface_pitch, surface_azimuth, lifespan, pv_eol_derating)
array_2 = SolarPVArray(1, 35, -45, 25, 0.88)
array_3 = SolarPVArray(1, 35, 0, 25, 0.88)
array_4 = SolarPVArray(1, 35, 45, 25, 0.88)
array_5 = SolarPVArray(1, 35, 90, 25, 0.88)
arrays = [array_1, array_2, array_3, array_4, array_5]

# Set up and run model
pv_model = SolarPVModel(site=site, arrays=arrays)

2024-02-15 23:14:43,577 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -90deg, Lifespan: 25yrs
2024-02-15 23:14:43,578 - INFO - *******************
2024-02-15 23:14:43,579 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -45deg, Lifespan: 25yrs
2024-02-15 23:14:43,579 - INFO - *******************
2024-02-15 23:14:43,579 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 0deg, Lifespan: 25yrs
2024-02-15 23:14:43,580 - INFO - *******************
2024-02-15 23:14:43,580 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 45deg, Lifespan: 25yrs
2024-02-15 23:14:43,581 - INFO - *******************
2024-02-15 23:14:43,581 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 90deg, Lifespan: 25yrs
2024-02-15 23:14:43,582 - INFO - *******************
2024-02-15 23:14:43,583 - INFO - *******************
2024-02-15 23:14:43,583 - INFO - Starting model calculations for SolarPVModel.
2024-02-15 23:14:43,584 - INFO -

# ***Visualisation***

In [28]:
import plotly.express as px
import seaborn as sns

pv_model.plot_sum(["Cell_Temp_C_Avg", "T2m"], group="daily", plot_type="bar")

In [8]:
pv_model.array_model(1).head()

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP,...,E_Diffuse_kWm2,E_Ground_kWm2,E_POA_kWm2,Panel_POA_kWm2,IAM_Loss_kWm2,ET_HRad_kWm2,PV_Derated_Eff,Cell_Temp_C,PV_Gen_kWh,PV_Thermal_Loss_kWh
0,20090101:0000,3.07,87.29,0.0,0.0,0.0,306.29,5.70,102.0,102898.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3.07,0.0,0.0
1,20090101:0100,2.63,86.63,0.0,0.0,0.0,299.79,5.73,102.0,102901.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.999999,2.63,0.0,0.0
2,20090101:0200,2.18,85.98,0.0,0.0,0.0,293.29,5.77,101.0,102905.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.999999,2.18,0.0,0.0
3,20090101:0300,1.73,85.32,0.0,0.0,0.0,286.79,5.80,103.0,102898.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.999998,1.73,0.0,0.0
4,20090101:0400,1.28,84.66,0.0,0.0,0.0,280.30,5.84,104.0,102891.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.999998,1.28,0.0,0.0


In [27]:
print(f"PV Gen: {round(pv_model.all_models.PV_Gen_kWh_Array_1.sum(), 3)}kWh")
print(f"E POA: {round(pv_model.all_models.E_POA_kWm2_Array_1.sum(), 3)}kWh")

PV Gen: 841.858kWh
E POA: 869.123kWh


In [12]:
px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [13]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [14]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [15]:
px.bar(pv_model.summary_grouped.daily[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [16]:
px.bar(pv_model.summary_grouped.hourly[["E_POA_kWm2_Total", "Panel_POA_kWm2_Total", "PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [17]:
px.bar(pv_model.summary_grouped.monthly[["E_POA_kWm2_Total", "Panel_POA_kWm2_Total", "PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [18]:
px.line(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])


In [19]:
px.line(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

# ***Mass Model Testing***

In [ ]:
# %%timeit
import random

# Assuming the Site, SolarPVArray, and SolarPVModel classes are defined elsewhere
site = Site(name="Test Site", address="123 Test Street", client="Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east=0)

surface_pitch = 35
azimuth_list = [90, 45, 0, -45, -90]
pv_eol = 0.88
cost = 1250
lifespan = 25
eff = 0.21
temp_coeff = -0.0048
t_ab = 0.9

max_pv = 7.5
min_pv = 2.5
pv_step_size = 0.25
pv_list = []
pv_models = {}

for i in range(int((max_pv - min_pv) / pv_step_size)):
    pv_list.append(min_pv + (i * pv_step_size))

for pv_size in pv_list:
    pv_kwp = pv_size

    array_1 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[0], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_2 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[1], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_3 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[2], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_4 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[3], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_5 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[4], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)

    # Initialize and run the SolarPVModel with all arrays
    arrays = [array_1, array_2, array_3, array_4, array_5]
    pv_model = SolarPVModel(site=site, arrays=arrays)

    pv_models[pv_size] = pv_model  # Store the model with pv_size as the key

Total_gen = 0

for model in pv_models.values():
    Total_gen += model.summary.PV_Gen_kWh_Annual

print("")
print("***************************************************")
print(f"{round(Total_gen / 1000, 3)} MWh from {len(pv_list)} models simulating {len(pv_list) * 5} PV arrays in total.")


In [ ]:
pv_models.get(2.5).models[0]["model_result"]

In [ ]:
# Access values from dictionary
pv_models.get(2.5).summary_grouped.daily.PV_Gen_kWh_Total.sum()

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.monthly.Cell_Temp_C_Avg - pv_models.get(2.5).summary_grouped.monthly.T2m)

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.weekly.PV_Gen_kWh_Total)

In [ ]:
sns.barplot(data=pv_models.get(2.5).summary_grouped.daily, x="Day_of_Year", y="PV_Gen_kWh_Total", 
            hue="PV_Gen_kWh_Total", palette="viridis")

In [ ]:
for pv_size in pv_list:
    print(f"{round(pv_models.get(pv_size).summary_grouped.daily.PV_Gen_kWh_Total.sum() / 1000, 3)} MWh/yr")

# ***VISUAL TESTING***

In [ ]:
variables = ['E_Beam_Wm2', 'E_Diffuse_Wm2', 'E_Ground_Wm2', 'E_POA_Wm2', 'ET_HRad_Wm2', 
            'Cell_Temp_C', 'PV_Gen_kWh', 'AOI', 'Zenith_Angle']

testy = pv_model.all_models

variable = "PV_Gen_kWh"

variable_1 = f"{variable}_Array_1"
variable_2 = f"{variable}_Array_2"
variable_3 = f"{variable}_Array_3"
variable_4 = f"{variable}_Array_4"
variable_7 = f"{variable}_Total"
variable_8 = f"{variable}_Avg"

if variable_1 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_1].mean())
if variable_2 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_2].mean())
if variable_3 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_3].mean())
if variable_4 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_4].mean())
if variable_7 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_7].mean(), alpha=0.4, ls="--")
if variable_8 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_8].mean(), alpha=0.4, ls="--")

In [ ]:
sns.lineplot(pv_model.combined_model["Cell_Temp_C_Avg"], alpha = 0.2, ls = "--", color = "orange")
sns.lineplot(pv_model.combined_model["T2m"])

In [ ]:
sns.barplot(pv_model.summary_grouped.daily.T2m)

In [ ]:
sns.lineplot(pv_model.combined_model.groupby("Hour_of_Day")["PV_Gen_kWh_Total"].mean())

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
testytest = pv_model.all_models


In [ ]:
filtered_columns = [col for col in testytest.columns if col.startswith('PV_Gen_kWh_Array_')]

In [ ]:
testytesty = testytest[filtered_columns].sum()

In [ ]:
testytesty[testytesty == testytesty.min()]

In [ ]:
pv_model.all_models.columns

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
sns.lineplot(pv_model.all_models["PV_Thermal_Loss_kWh_Total"])

In [ ]:
temp_losses = (pv_model.all_models["PV_Thermal_Loss_kWh_Total"].sum() / pv_model.all_models["PV_Gen_kWh_Total"].sum() * 100)

print(f"{round(temp_losses, 3)}% change to power due to temperature losses")

In [ ]:
pv_model.all_models

# ***MODEL ANALYSIS***